In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

In [3]:
train = train.rename(columns = {'Warehouse_block ': 'Warehouse_block'})
test = test.rename(columns = {'Warehouse_block ': 'Warehouse_block'})

In [4]:
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Roadzk', 'Road')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Ship', 'Ship')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Flight', 'Flight')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Road', 'Road')

In [6]:
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Roadzk', 'Road')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Ship', 'Ship')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Flight', 'Flight')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Road', 'Road')

In [7]:
train['Customer_rating']=train['Customer_rating'].replace(99, 3)
test['Customer_rating']=test['Customer_rating'].replace(99, 3)

In [8]:
train['Product_importance'] = train['Product_importance'].replace('mediumm', 'medium')
train['Product_importance'] = train['Product_importance'].replace('loww', 'low')
train['Product_importance'] = train['Product_importance'].replace('highh', 'high')

In [9]:
test['Product_importance'] = test['Product_importance'].replace('mediumm', 'medium')
test['Product_importance'] = test['Product_importance'].replace('loww', 'low')
test['Product_importance'] = test['Product_importance'].replace('highh', 'high')

In [10]:
train['Product_importance'] = train['Product_importance'].replace('?','low')
test['Product_importance'] = test['Product_importance'].replace('?','low')

In [11]:
train['Customer_care_calls']=train['Customer_care_calls'].fillna(2)
test['Customer_care_calls']=test['Customer_care_calls'].fillna(2)

In [12]:
train['Customer_care_calls']=train['Customer_care_calls'].astype('int64')
test['Customer_care_calls']=test['Customer_care_calls'].astype('int64')

In [13]:
train['Prior_purchases']=train['Prior_purchases'].fillna(0)
test['Prior_purchases']=test['Prior_purchases'].fillna(0)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999 entries, 1 to 6999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      6999 non-null   object 
 1   Mode_of_Shipment     6999 non-null   object 
 2   Customer_care_calls  6999 non-null   int64  
 3   Customer_rating      6999 non-null   int64  
 4   Cost_of_the_Product  6999 non-null   int64  
 5   Prior_purchases      6999 non-null   float64
 6   Product_importance   6999 non-null   object 
 7   Gender               6999 non-null   object 
 8   Discount_offered     3468 non-null   float64
 9   Weight_in_gms        6999 non-null   object 
 10  Reached.on.Time_Y.N  6999 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 656.2+ KB


In [17]:
train['Discount_offered']=train['Discount_offered'].fillna(0)
test['Discount_offered']=test['Discount_offered'].fillna(0)

In [18]:
mask = train['Weight_in_gms'].isin(['?'])
del_wei_question=train[~mask]

In [20]:
del_wei_question['Weight_in_gms']=del_wei_question['Weight_in_gms'].astype('int64')

<ipython-input-20-9102d6bf3727>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_wei_question['Weight_in_gms']=del_wei_question['Weight_in_gms'].astype('int64')


In [21]:
pivot = del_wei_question.pivot_table(values='Weight_in_gms',
                         index=['Customer_care_calls'],
                         aggfunc='mean'
                         )
pivot['Weight_in_gms']=pivot['Weight_in_gms'].astype('int64')

In [22]:
train['Weight_in_gms']=train['Weight_in_gms'].replace('?', '0')

In [23]:
test['Weight_in_gms']=test['Weight_in_gms'].replace('?', '0')

In [24]:
train['Weight_in_gms']=train['Weight_in_gms'].astype('int64')

In [25]:
test['Weight_in_gms']=test['Weight_in_gms'].astype('int64')

In [26]:
def fill_wei(data):
    if data['Weight_in_gms']==0:
        return pivot.loc[data['Customer_care_calls']]
    else:
        return data['Weight_in_gms']

In [27]:
train['Weight_in_gms'] = train.apply(fill_wei, axis=1).astype('int64')

In [28]:
test['Weight_in_gms'] = test.apply(fill_wei, axis=1).astype('int64')